## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [1]:
import os
from utils import setup_environment, load_data, setup_logging

from retriever import retriever
from planner import planner
from dreamer import dreamer
from grounder import grounder
from executor import executor

from schemas import RetrieverOut, PlannerOut, DreamerOut

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне
db = db.sample(1000, random_state=42)               # Ограничение по числу строк - уменьшаем контексты

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [2]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        # "model": "deepseek/deepseek-chat-v3.1",
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.8,
        "reasoning_effort": None
    },
    # параметры аналитика-выдумщика
    dreamer_params={
        # "model": "deepseek/deepseek-chat-v3.1",
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 1,
        "reasoning_effort": "high"
    },
    # параметры планировщика
    planner_params={
        "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.2
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Logger

Настройка логирования и директорий сохранения файлов

In [3]:
run_dir = setup_logging(mode="INFO", log_dir="logs")

retriever_out_path = os.path.join(run_dir, "retrieved.json")
dreamer_out_path = os.path.join(run_dir, "dreamed.json")
planner_out_path = os.path.join(run_dir, "planned.json")

Logging to file: logs\run_2025-11-14_14-30-43\log.txt


### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [4]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
# user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
user_query = "Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [5]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

retriever_out = retriever(user_query, PPL_cfg)
retriever_out.save(retriever_out_path)

2025-11-14 14:30:57 | INFO     | retriever            | Starting retriever for query: Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть...
2025-11-14 14:31:19 | INFO     | retriever            | Retriever completed with 7 questions


Можно посмотреть reasoning, если модель его поддерживает и он был указан в параметрах

In [ ]:
# print(retriever_out.reasoning)

In [6]:
retriever_out = RetrieverOut.load(retriever_out_path)
print(retriever_out)

1. '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?'
	Reason: Данный вопрос предоставляет интервалы размера сбережений, необходимые для расчета средних значений.
2. '[Family_income_new] К какому из следующих интервалов Вы могли бы отнести месячный доход ВАШЕЙ СЕМЬИ (доход членов семьи, проживающих в одной квартире)?'
	Reason: Доходы необходимы для вычисления нормы сбережений (соотношение сбережений к доходу).
3. '[N4] Какой % от Вашего дохода Вы сберегаете ежемесячно?'
	Reason: Прямой показатель нормы сбережений (%D), но только для тех, кто сберегает. Ответы от 0-9% позволяют выделить аудиторию с активными сбережениями.
4. '[Y1nn] К какому из следующих интервалов Вы относите свою месячную зарплату?'
	Reason: Дает данные о доходе, чтобы рассчитать абсолютную сумму сбережений (например, 30-39 тыс. руб. × 0-9%).
5. '[N41_main_second_1] Какой банк для Вас второй после основого? по сумме средств в банке'
	Reason: Включает конкретные су

### Dreamer

Строит план анализа без привязки к функционалу

In [7]:
dreamer_out = dreamer(user_query, PPL_cfg)
dreamer_out.save(dreamer_out_path)

In [8]:
dreamer_out = DreamerOut.load(dreamer_out_path)
print(dreamer_out)

####################
АНАЛИЗ
####################


**План анализа для расчёта средних сбережений и нормы сбережений:**

1. **Анализ сбережений ([A26]):**
   - Преобразовать интервалы сбережений в средние значения:
     - `400,000-700,000 рублей` → Среднее: 550,000 рублей.
     - `60,000-90,000 рублей` → Среднее: 75,000 рублей.
   - Рассчитать средние сбережения как среднее этих значений:
     $$
     \text{Средние сбережения} = \frac{550,000 + 75,000}{2} = 312,500 \, \text{рублей}.
     $$

2. **Анализ нормы сбережений ([N4]):**
   - Выбрать интервалы, где дано значение `0-9%` (единый ответ в данных).
   - Рассчитать среднюю норму как среднее интервала:
     $$
     \text{Норма сбережений} = \frac{0 + 9}{2} = 4.5\%.
     $$
   - Это значение отражает средний процент сбережений от дохода среди тех, кто учитывается в данных.

**Ограничения и необходимость уточнений:**
   - Отсутствие весов (количества респондентов в каждом интервале) может искажать результаты. Необходимо проверить данные

Можно отдельно посмотреть **анализ** и **reasoning**, раскомментировав соответствующую строку

In [10]:
# print(dreamer_out.analysis)
# print(dreamer_out.reasoning)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [13]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, dreamer_out, PPL_cfg)
planner_out.save(planner_out_path)

2025-11-14 14:36:28 | INFO     | planner              | Starting planner for query: Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть...
2025-11-14 14:36:36 | INFO     | planner              | Planner completed: 4 steps generated


In [14]:
planner_out = PlannerOut.load(planner_out_path)
print(planner_out)

АНАЛИЗ: Для выполнения задачи по расчёту средних сбережений и нормы сбережений мне необходимо выполнить следующие шаги:

1. Загрузить данные из доступных волн опроса
2. Провести анализ распределения ответов по вопросу о размере сбережений ([A26])
3. Провести анализ распределения ответов по вопросу о норме сбережений ([N4])
4. Возможно, провести дополнительные проверки с использованием других вопросов (например, дохода)

Для расчёта средних сбережений мне нужно знать количество респондентов в каждом интервале, чтобы правильно усреднить значения. В текущем плане указано, что если 70% имеют 60-90 тыс. рублей, а 30% - 400-700 тыс., то средние сбережения будут 202,500 рублей. Это означает, что для точного расчёта мне нужно знать доли респондентов в каждом интервале.

Для расчёта нормы сбережений мне также нужно знать, что интервал 0-9% в вопросе [N4] включает всех респондентов, включая тех, кто не сберегает. Если требуется только для тех, у кого есть сбережения, нужно будет исключить тех, к

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [15]:
grounder_out = grounder(planner_out)

2025-11-14 14:36:59 | INFO     | grounder             | Starting grounder
2025-11-14 14:36:59 | INFO     | grounder             | Grounder completed: 4 steps grounded


### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [16]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

2025-11-14 14:37:05 | INFO     | executor             | Starting executor with 4 steps
2025-11-14 14:37:05 | INFO     | executor             | Steps execution order: ['s1', 's2', 's3', 's4']
2025-11-14 14:37:05 | INFO     | executor             | Executing step 1/4: s1 (LOAD_DATA)
2025-11-14 14:37:05 | INFO     | executor             | Step s1 completed successfully
2025-11-14 14:37:05 | INFO     | executor             | Executing step 2/4: s2 (PIVOT)
2025-11-14 14:37:05 | INFO     | executor             | Step s2 completed successfully
2025-11-14 14:37:05 | INFO     | executor             | Executing step 3/4: s3 (PIVOT)
2025-11-14 14:37:05 | INFO     | executor             | Step s3 completed successfully
2025-11-14 14:37:05 | INFO     | executor             | Executing step 4/4: s4 (PIVOT)
2025-11-14 14:37:05 | INFO     | executor             | Step s4 completed successfully
2025-11-14 14:37:05 | INFO     | executor             | Executor completed successfully


Далее можно извлекать созданные таблицы из `ctx`

In [17]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Доступные варианты:
1. 'dataset'
2. 'pivot'
3. 'pivot_A26'
4. 'pivot_N4'
5. 'pivot_N46'


Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["pivot_N46"]